# [RAG 문제]
- law_2.docx 파일을 읽고 Chroma 저장  
- LLM 질문 - > 답변
- 전세사기피해에 관한 법률 질문만 받도록.  
- 이외의 질문은 '답변을 할 수 없습니다.'라고 답변.  

In [35]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from dotenv import load_dotenv

load_dotenv()


loader = Docx2txtLoader("law_2.docx")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, 
    chunk_overlap=150,
)

data_split = loader.load_and_split(text_splitter=text_splitter)

embedding = OpenAIEmbeddings(
    model="text-embedding-3-large",
)


In [51]:

# database = Chroma.from_documents(
#     documents=data_split,
#     embedding=embedding,
#     persist_directory='./chroma',
#     collection_name='chroma-law2',
# )

database = Chroma(
    embedding_function=embedding,
    persist_directory='./chroma',                  
    collection_name='chroma-law2',
)

    
query = input("질문을 입력하세요. 관련 없는 질문은 답변을 할 수 없습니다.")

retrieved_data = database.similarity_search(query=query, k=2)

retrieved_data

[Document(id='622503ab-fb24-4044-975b-9a7fed01cf66', metadata={'source': 'law_2.docx'}, page_content='제4조의2(전세사기피해주택을 매입하여 우선 공급하는 경우의 재정 지원) ① 법 제25조제6항에 따른 차액을 산정하는 경우 같은 조 제4항에 따라 전세사기피해주택을 취득하는 데 소요되는 비용은 다음 각 호의 비용으로 한다.\n\n  1. 「민사집행법」에 따른 경매, 「국세징수법」 또는 「지방세징수법」에 따른 공매 절차에서 「공공주택 특별법」 제4조제1항제2호 또는 제3호에 따른 공공주택사업자(이하 “공공주택사업자”라 한다)가 납부하는 매각대금 또는 매수대금\n\n  2. 전세사기피해주택의 공용부분에 대한 전세사기피해자의 미납 사용료 및 관리비\n\n  ② 법 제25조제6항에 따른 차액을 산정하는 경우 「공공주택 특별법」 제43조에 따라 매입하였을 때 소요되었을 금액은 「감정평가 및 감정평가사에 관한 법률」에 따른 감정평가법인등(이하 “감정평가법인등”이라 한다) 2인이 평가한 금액의 산술평균치로 한다. 이 경우 감정평가법인등은 공공주택사업자의 요청에 따라 「감정평가 및 감정평가사에 관한 법률」 제33조에 따른 한국감정평가사협회가 추천한 4인 이상의 감정평가법인등 중에서 공공주택사업자와 전세사기피해자가 각각 1인씩 선정한다.\n\n  ③ 국가 또는 지방자치단체는 법 제25조제7항에 따라 임대료를 재정으로 지원하는 경우에는 전세사기피해자가 같은 조 제4항에 따른 거주기간 동안 납부해야 하는 임대료에서 같은 조 제6항에 따른 차액을 공제한 금액을 전액 지원한다. 다만, 공공주택사업자가 전세사기피해자의 요청에 따라 제2항 전단에 따른 산술평균치로 전세사기피해주택을 매입한 경우로서 제1호의 금액이 제2호의 금액보다 큰 경우에는 제1호의 금액에서 제2호의 금액을 뺀 금액을 지원하지 않을 수 있다.\n\n  1. 전세사기피해주택 매입금액\n\n  2. 전세사기피해주택이 소재하는 지역의 주택으로서 

In [52]:
## 문서 객체 추출 [방법1]
# ex = "" # 빈 문자열

# for doc in retrieved_data:
#     # print(doc.page_content + "△△△△△△△△△△△\n")
#     ex += doc.page_content +'~~~~~~~~~~~~~~~~~' ## 문자열 + 는 연속해서 들어옴.



##문서 객체 추출 [방법2] 리스트내포
contents = '\n\n\n'.join([doc.page_content for doc in retrieved_data])

# 밑 프롬프트에 {이 안에} 문자열만 포함되게 하는 방식임. 

contents

'제4조의2(전세사기피해주택을 매입하여 우선 공급하는 경우의 재정 지원) ① 법 제25조제6항에 따른 차액을 산정하는 경우 같은 조 제4항에 따라 전세사기피해주택을 취득하는 데 소요되는 비용은 다음 각 호의 비용으로 한다.\n\n  1. 「민사집행법」에 따른 경매, 「국세징수법」 또는 「지방세징수법」에 따른 공매 절차에서 「공공주택 특별법」 제4조제1항제2호 또는 제3호에 따른 공공주택사업자(이하 “공공주택사업자”라 한다)가 납부하는 매각대금 또는 매수대금\n\n  2. 전세사기피해주택의 공용부분에 대한 전세사기피해자의 미납 사용료 및 관리비\n\n  ② 법 제25조제6항에 따른 차액을 산정하는 경우 「공공주택 특별법」 제43조에 따라 매입하였을 때 소요되었을 금액은 「감정평가 및 감정평가사에 관한 법률」에 따른 감정평가법인등(이하 “감정평가법인등”이라 한다) 2인이 평가한 금액의 산술평균치로 한다. 이 경우 감정평가법인등은 공공주택사업자의 요청에 따라 「감정평가 및 감정평가사에 관한 법률」 제33조에 따른 한국감정평가사협회가 추천한 4인 이상의 감정평가법인등 중에서 공공주택사업자와 전세사기피해자가 각각 1인씩 선정한다.\n\n  ③ 국가 또는 지방자치단체는 법 제25조제7항에 따라 임대료를 재정으로 지원하는 경우에는 전세사기피해자가 같은 조 제4항에 따른 거주기간 동안 납부해야 하는 임대료에서 같은 조 제6항에 따른 차액을 공제한 금액을 전액 지원한다. 다만, 공공주택사업자가 전세사기피해자의 요청에 따라 제2항 전단에 따른 산술평균치로 전세사기피해주택을 매입한 경우로서 제1호의 금액이 제2호의 금액보다 큰 경우에는 제1호의 금액에서 제2호의 금액을 뺀 금액을 지원하지 않을 수 있다.\n\n  1. 전세사기피해주택 매입금액\n\n  2. 전세사기피해주택이 소재하는 지역의 주택으로서 전세사기피해주택과 면적 및 권리관계 등이 유사한 주택의 평균 낙찰가격 등을 고려하여 국토교통부장관이 정하는 기준에 따라 산정한 금액\n\n\n2. 상속세, 증여세 및 종합부동산세 외

In [53]:

prompt = '''
[identity]
- 당신은 전세사기피해 관련 법률 전문가입니다. 
- [context]을 참고해 사용자의 질문에 답변하세요. 
- [context]에 없는 내용에 대해서는 "관련 내용만 질문해주세요."라고 대답하세요. 

[context]
{retrieved_data}

[question]
{query}

'''

llm = ChatOpenAI(
    model_name='gpt-4.1-nano',
)

aimessage = llm.invoke(
    prompt.format(
        retrieved_data=contents,
        query=query
    )
)

aimessage.content

'전세사기피해주택 매입금액은 다음과 같이 정해져 있습니다.\n\n1. 「민사집행법」에 따른 경매, 「국세징수법」 또는 「지방세징수법」에 따른 공매 절차에서 공공주택사업자가 납부하는 매각대금 또는 매수대금입니다.\n\n2. 전세사기피해주택의 공용부분에 대한 전세사기피해자의 미납 사용료 및 관리비입니다.\n\n또한, 법 제25조의2(전세사기피해주택을 매입하여 우선 공급하는 경우의 재정 지원)에서는 감정평가법인등이 평가한 금액의 산술평균치를 기준으로 산정할 수도 있음을 규정하고 있습니다.'

In [43]:
while True :
    query = input("질문을 입력하세요. 관련 없는 질문은 답변을 할 수 없습니다. S을 입력하면 대화 종료. ")
    if query.upper() == "S" :
        break

    aimessage = llm.invoke(
    prompt.format(
        retrieved_data=contents,
        query=query
        )
    )
    print(query)
    print(aimessage.content)
    print('*'*50)

 

전세사기 피해자는 누구인가요?
관련 내용만 질문해주세요.
**************************************************
전세사기 피해자 구제 방법을 알려주세요
관련 내용만 질문해주세요.
**************************************************
 s
관련 내용만 질문해주세요.
**************************************************
